In [3]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd

# Correctly load the data from CSV
df = pd.read_csv("CleanedData/player_stats_cleaned.csv")  # Make sure the path is correct

# Build App
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col(html.Div([
                    dbc.Card(
                        dbc.CardBody([
                            html.Div([
                                html.H2("Select Players"),
                            ], style={'textAlign': 'center'})
                        ])
                    ),
                ]), width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='player-dropdown',
                        options=[{'label': i, 'value': i} for i in df['player'].unique()],
                        multi=True,
                        placeholder="Select players",
                        style={"color": "#000"}
                    )
                ], width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col(dcc.Graph(id='stacked-bar-chart'), width=12),
            ], align='center'),
        ]), color='dark'
    )
])

@app.callback(
    Output('stacked-bar-chart', 'figure'),
    [Input('player-dropdown', 'value')]
)
def update_graph(selected_players):
    if not selected_players:
        return go.Figure()

    fig = go.Figure()
    # Control the legend entries
    first_player = True  # Flag to show legend only for the first player
    for player in selected_players:
        filtered_df = df[df['player'] == player]
        goals_sum = filtered_df['goals'].sum()
        assists_sum = filtered_df['assists'].sum()

        fig.add_trace(go.Bar(
            name='Goals',
            x=[player],
            y=[goals_sum],
            marker_color='red',
            showlegend=first_player  # Show legend only for the first player
        ))
        fig.add_trace(go.Bar(
            name='Assists',
            x=[player],
            y=[assists_sum],
            marker_color='blue',
            showlegend=first_player  # Show legend only for the first player
        ))
        first_player = False  # Update flag after the first player

    fig.update_layout(
        barmode='stack',
        title='Player Goals and Assists',
        xaxis_title='Player',
        yaxis_title='Count',
        legend_title='Metric'
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port= 8054)


In [4]:
import threading
from IPython.display import display, HTML

# Function to run the Dash app
def run_dash():
    app.run_server(debug=False, use_reloader=False)

# Start the Dash app in a separate thread
threading.Thread(target=run_dash).start()

# Display the link to open the app in the browser
display(HTML(f"""
    <a href="http://127.0.0.1:8054/" target="_blank">
        Open Dash App
    </a>
"""))